In [12]:
import os
import sys; sys.path.append("~/HHbbgg_ETH_devel/bregression/python") # to load packages
import training_utils as utils
import numpy as np
reload(utils)
import preprocessing_utils as preprocessing
reload(preprocessing)
import plotting_utils as plotting
reload(plotting)
import optimization_utils as optimization
reload(optimization)
import postprocessing_utils as postprocessing
reload(postprocessing)

<module 'postprocessing_utils' from '/mnt/t3nfs01/data01/shome/nchernya/HHbbgg_ETH_devel/bregression/python/postprocessing_utils.pyc'>

In [13]:
ntuples = 'heppy_05_10_2017'
# "%" sign allows to interpret the rest as a system command
%env data=$utils.IO.ldata$ntuples
files = ! ls $data | sort -t_ -k 3 -n

#ttbar= [s for s in files if "20trees_safe" in s] #large statistics
ttbar= [s for s in files if "ttbar_RegressionPerJet.root" in s] # only limited statistics


utils.IO.add_target(ntuples,ttbar,1)
utils.IO.add_features(ntuples,ttbar,1)

for i in range(len(utils.IO.targetName)):        
    print "using target file n."+str(i)+": "+utils.IO.targetName[i]
for i in range(len(utils.IO.featuresName)):        
    print "using features file n."+str(i)+": "+utils.IO.featuresName[i]

env: data=/mnt/t3nfs01/data01/shome/nchernya/HHbbgg_ETH_devel/root_files/heppy_05_10_2017
using target file n.0: /mnt/t3nfs01/data01/shome/nchernya/HHbbgg_ETH_devel/root_files/heppy_05_10_2017/ttbar_RegressionPerJet.root
using features file n.0: /mnt/t3nfs01/data01/shome/nchernya/HHbbgg_ETH_devel/root_files/heppy_05_10_2017/ttbar_RegressionPerJet.root


In [22]:
#use noexpand for root expressions, it needs this file https://github.com/ibab/root_pandas/blob/master/root_pandas/readwrite.py
#noexpand:fabs(CosThetaStar_CS)
branch_names = 'Jet_pt,noexpand:Jet_mcPt/Jet_pt,Jet_eta,Jet_corr,Jet_mcPt,Jet_mcFlavour,dR,rho,Jet_mt,Jet_leadTrackPt,Jet_leptonPtRel,Jet_leptonPt,Jet_leptonDeltaR,Jet_neHEF,Jet_neEmEF,Jet_chMult,Jet_vtxPt,Jet_vtxMass,Jet_vtx3dL,Jet_vtxNtrk,Jet_vtx3deL'.split(",")

features = 'Jet_pt,Jet_eta,Jet_corr,rho,Jet_mt,Jet_leadTrackPt,Jet_leptonPtRel,Jet_leptonPt,Jet_leptonDeltaR,Jet_neHEF,Jet_neEmEF,Jet_vtxPt,Jet_vtxMass,Jet_vtx3dL,Jet_vtxNtrk,Jet_vtx3deL'.split(",")
#target = 'Jet_mcPt'.split(",")
target = 'Jet_mcPt/Jet_pt'.split(",")
#cuts='Jet_mcPt > 20 and dR < 0.35 and (Jet_mcFlavour==5 or Jet_mcFlavour==-5) and Jet_pt > 15'
cuts='(Jet_mcPt > 20) & (Jet_mcFlavour==5 | Jet_mcFlavour==-5) & (Jet_pt > 15) & (Jet_eta<2.4 & Jet_eta>-2.4) & (dR < 0.4)'



branch_names = [c.strip() for c in branch_names]
features = [c.strip() for c in features]
target = [c.strip() for c in target]

#print branch_names

import pandas as pd
import root_pandas as rpd


X_features = preprocessing.set_features("tree",branch_names,features,cuts)
X_target = preprocessing.set_target("tree",branch_names,target,cuts)
X_features,X_target=preprocessing.randomize_ft(X_features,X_target)

#print max(X_features[:,1])

#print X_target
#print pd.DataFrame(utils.IO.features_df[0],columns=['Jet_eta'])

X_train_target = preprocessing.get_training_sample(pd.DataFrame(X_target))
X_train_features = preprocessing.get_training_sample(pd.DataFrame(X_features))

X_test_target = preprocessing.get_test_sample(pd.DataFrame(X_target))
X_test_features = preprocessing.get_test_sample(pd.DataFrame(X_features))

#X_target_data =  pd.DataFrame(utils.IO.data_df[0],columns=['Jet_mcPt'])
#X_data_target = utils.IO.data_df[0][['Jet_mcPt']]
#new_jet_mcPt = jet_mcPt.query('Jet_mcPt > 30')

from sklearn.externals import joblib
import xgboost as xgb
import matplotlib.pyplot as plt

X_all = preprocessing.cut_region("tree",branch_names,branch_names,cuts)

pt_regions = '(Jet_mcPt<100),(Jet_mcPt>=100 & Jet_mcPt<300),(Jet_mcPt>=300 & Jet_mcPt<1000),(Jet_mcPt>1000)'.split(",")
X_pt_region=[]
target_dist = []
target_dist.append('noexpand:Jet_mcPt/Jet_pt')
for region in pt_regions:
    cuts_regions = cuts+'&'+region
    X_pt_region.append(preprocessing.cut_region("tree",branch_names,target_dist,cuts_regions))

#log_names='Jet_pt,Jet_mcPt,Jet_mt,Jet_leadTrackPt,Jet_leptonPtRel,Jet_leptonPt,Jet_leptonDeltaR'.split(",")
#plotting.plot_input_variables_reg(X_features,features,log_names)
#plotting.plot_input_variables_reg(X_all,branch_names,log_names)
#plt.show()






(Jet_mcPt > 20) & (Jet_mcFlavour==5 | Jet_mcFlavour==-5) & (Jet_pt > 15) & (Jet_eta<2.4 & Jet_eta>-2.4) & (dR < 0.4)&(Jet_mcPt<100)
(Jet_mcPt > 20) & (Jet_mcFlavour==5 | Jet_mcFlavour==-5) & (Jet_pt > 15) & (Jet_eta<2.4 & Jet_eta>-2.4) & (dR < 0.4)&(Jet_mcPt>=100 & Jet_mcPt<300)
(Jet_mcPt > 20) & (Jet_mcFlavour==5 | Jet_mcFlavour==-5) & (Jet_pt > 15) & (Jet_eta<2.4 & Jet_eta>-2.4) & (dR < 0.4)&(Jet_mcPt>=300 & Jet_mcPt<1000)
(Jet_mcPt > 20) & (Jet_mcFlavour==5 | Jet_mcFlavour==-5) & (Jet_pt > 15) & (Jet_eta<2.4 & Jet_eta>-2.4) & (dR < 0.4)&(Jet_mcPt>1000)
[        Jet_mcPt/Jet_pt
0               0.90514
6               1.07681
23              1.05396
25              0.94038
29              1.31530
34              1.42335
37              0.96713
45              1.06035
46              0.94843
59              0.81392
69              1.30739
71              1.57551
76              1.49895
82              0.93861
90              1.16300
91              1.13801
97              0.87679
100  

In [5]:
########final optimization with all fixed#######

from sklearn.externals import joblib
import xgboost as xgb
from sklearn.metrics import confusion_matrix, mean_squared_error
from sklearn.grid_search import GridSearchCV


#clf = xgb.XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
    #   gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=9,
    #   min_child_weight=1e-05, missing=None, n_estimators=1500, nthread=20,
    #   objective='reg:linear', reg_alpha=0, reg_lambda=0.1,
    #   scale_pos_weight=1, seed=0, silent=True, subsample=1)

clf = xgb.XGBRegressor(objective='reg:linear')
#param_grid = {'n_estimators': [100,500]}
#grid_scores = optimization.optimize_parameters_gridCV_ref(xgb_model,X_train_features,X_train_target,X_test_features,X_test_target,param_grid)

#clf = GridSearchCV(xgb_model, {'max_depth': [4,5,6]}, verbose=1,n_jobs==-1)
clf.fit(X_train_features,X_train_target)
#print clf.best_score_
#print clf.best_params_
#joblib.dump(clf, os.path.expanduser('~/HHbbgg_ETH_devel/bregression/output_files/regression_heppy_mcPt_cuts.pkl'), compress=9)
predictions = clf.predict(X_test_features)
#print predictions, predictions.shape
actuals = X_test_target
true = (actuals.as_matrix()).ravel()
recoPt = (X_test_features[0].as_matrix()).ravel()
##print true, true.shape
#print true[0]/predictions[0]
ratio = true/predictions
ratio=1/ratio
reco_ratio=true/recoPt
reco_ratio=1./reco_ratio
#print ratio.shape
#print ratio


#print(mean_squared_error(actuals, predictions))
#print type(actuals), type(predictions)



/swshare/anaconda/lib/python2.7/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [ ]:
import matplotlib.pyplot as plt
import training_utils as utils
import ROOT
from ROOT import gROOT
#import plotting_utils as plotting
#import training_utils as utils
reload(utils)
reload(plotting)

regressed_pt = predictions*recoPt
true_pt = true*recoPt

#xgb.plot_importance(clf)
#plt.show()
#plt.savefig(utils.IO.plotFolder+"importance1.pdf")

print 'I am doing something here'
style=False
#plotting.plot_rel_pt_diff(predictions,true,recoPt,style,100,)
#plotting.plot_rel_pt_diff(regressed_pt,true_pt,recoPt,style,100,)


In [11]:
cuts0='(Jet_mcPt > 20) & Jet_eta>-2.4) & (dR < 0.4)'
cut2 = '& (Jet_eta > 1.2)'
cutnew = cuts0+cut2
print cutnew

(Jet_mcPt > 20) & Jet_eta>-2.4) & (dR < 0.4)& (Jet_eta > 1.2)
